In [1]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os

In [ ]:
!unzip Confirmed_fronts.zip -d my_data

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Tiguan$$2008$$Black$$95_26$$559$$image_4.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Tiguan$$2008$$Grey$$95_26$$742$$image_0.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touareg$$2008$$Black$$95_28$$333$$image_5.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touareg$$2008$$Grey$$95_28$$269$$image_5.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touareg$$2008$$Silver$$95_28$$563$$image_5.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touran$$2008$$Grey$$95_29$$1397$$image_5.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touran$$2008$$Grey$$95_29$$48$$image_3.jpg  
  inflating: my_data/confirmed_fronts/Volkswagen/2008/Volkswagen$$Touran$$2008$$Unlisted$$95_29$$62$$image_1.jpg  
  inflating: my_data/confi

In [ ]:
import os
import shutil

def create_color_dataset(source_path, target_path):
    """
    Создает новый датасет с папками для каждого цвета.

    Args:
        source_path: Путь к исходному датасету.
        target_path: Путь к новому датасету.
    """

    os.makedirs(target_path, exist_ok=True)  # Создаем целевую папку, если ее нет

    for root, _, files in os.walk(source_path):
        for filename in files:
            filepath = os.path.join(root, filename)
            color = filename.split('$$')[3]  # Извлекаем цвет из имени файла
            color_path = os.path.join(target_path, color)
            os.makedirs(color_path, exist_ok=True)  # Создаем папку для цвета, если ее нет
            shutil.copy2(filepath, color_path)  # Копируем файл в папку цвета

if __name__ == '__main__':
    source_path = 'my_data'  # Путь к исходному датасету
    target_path = 'color_dataset'  # Путь к новому датасету

    create_color_dataset(source_path, target_path)
    print(f'Создан новый датасет: {target_path}')

Создан новый датасет: color_dataset


In [ ]:
dataset_path = 'color_dataset'  # Путь к новому датасету

colors = {}
total_files = 0

for root, _, files in os.walk(dataset_path):
    color = os.path.basename(root)  # Получаем имя цвета из папки
    if color != 'color_dataset':  # Проверяем, не корневая ли папка
        colors[color] = len(files)  # Добавляем количество файлов в цвет
        total_files += len(files)

print(f'Количество цветов: {len(colors)}')
print('Количество файлов в каждом цвете:')
for color, count in colors.items():
    print(f'  {color}: {count}')
print(f'Общее количество файлов: {total_files}')

Количество цветов: 23
Количество файлов в каждом цвете:
  Orange: 559
  Yellow: 667
  Brown: 911
  Green: 777
  Magenta: 9
  Maroon: 26
  Silver: 7770
  Beige: 600
  Gold: 217
  White: 9395
  Grey: 9474
  Blue: 8483
  Red: 6095
  Indigo: 1
  Navy: 1
  Pink: 87
  Multicolour: 196
  Turquoise: 26
  Burgundy: 9
  Purple: 362
  Unlisted: 1516
  Black: 14317
  Bronze: 329
Общее количество файлов: 61827


In [ ]:
import random

In [ ]:
def split_dataset(dataset_path, train_ratio=0.8):
    colors = os.listdir(dataset_path)
    train_path = os.path.join(dataset_path, 'train')
    test_path = os.path.join(dataset_path, 'test')

    os.makedirs(train_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

    for color in colors:
        color_path = os.path.join(dataset_path, color)
        files = os.listdir(color_path)
        random.shuffle(files)

        train_size = int(train_ratio * len(files))
        train_files = files[:train_size]
        test_files = files[train_size:]

        train_color_path = os.path.join(train_path, color)
        test_color_path = os.path.join(test_path, color)

        os.makedirs(train_color_path, exist_ok=True)
        os.makedirs(test_color_path, exist_ok=True)

        for filename in train_files:
            shutil.move(os.path.join(color_path, filename), train_color_path)
        for filename in test_files:
            shutil.move(os.path.join(color_path, filename), test_color_path)

if __name__ == '__main__':
    dataset_path = 'color_dataset'  # Путь к датасету

    split_dataset(dataset_path)
    print('Датасет разделен на train и test выборки.')

Датасет разделен на train и test выборки.


In [ ]:
from torchvision import transforms as T
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
img_sz = 224

normalize = T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_data_transforms = T.Compose([
        T.Resize((img_sz, img_sz)),
        T.RandomHorizontalFlip(),
        T.RandomRotation(degrees=30),
        T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        T.ToTensor(),
        normalize
    ])
test_data_transforms = T.Compose([
    T.Resize((img_sz, img_sz)),
    T.ToTensor(),
    normalize
])

In [ ]:
# !rm -rf /content/color_dataset/train/.ipynb_checkpoints

In [ ]:
train_dataset = datasets.ImageFolder(root='color_dataset/train', transform=train_data_transforms)
test_dataset = datasets.ImageFolder(root='color_dataset/test', transform=test_data_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)


        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = VGG11(num_classes=2, in_channels=3).to(device)

In [ ]:
!pip install torchsummary